In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig: 
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str

In [10]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories


In [ ]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig: 
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str



class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        print("alphabeta",self.params)
        config= self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([self.config.artifacts_root])
        
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )
        
        return model_trainer_config
        
    

In [16]:
import pandas as pd
import os
from src.mlops_pipeline import logger
from sklearn.linear_model import ElasticNet
import joblib

from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig: 
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str



class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        print("alphabeta",self.params)
        config= self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([self.root_dir])
        
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )
        
        return model_trainer_config
        
    


class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y= train_data[self.config.target_column]
        test_y = train_data[self.config.target_column]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        
        
STAGE_NAME = "Model Trainer stage"

try:
    config = ConfigurationManager()
    data_validation_config = config.get_model_trainer_config()
    data_validation = ModelTrainer(config=data_validation_config)
    data_validation.train()
    
except Exception as e:
    logger.exception(e)
    raise e

[2025-11-18 13:20:55,454: INFO: common :yaml file: config\config.yaml loaded]
[2025-11-18 13:20:55,465: INFO: common :yaml file: params.yaml loaded]
[2025-11-18 13:20:55,471: INFO: common :yaml file: schema.yaml loaded]
[2025-11-18 13:20:55,473: INFO: common :created directory at:artifacts]
alphabeta {'ElasticNet': {'alpha': 0.2, 'l1_ratio': 0.1}}
[2025-11-18 13:20:55,481: ERROR: 3949965930 :'ConfigurationManager' object has no attribute 'root_dir']
Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_27852\3949965930.py", line 80, in <module>
    data_validation_config = config.get_model_trainer_config()
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_27852\3949965930.py", line 38, in get_model_trainer_config
    create_directories([self.root_dir])
AttributeError: 'ConfigurationManager' object has no attribute 'root_dir'


AttributeError: 'ConfigurationManager' object has no attribute 'root_dir'

In [9]:


STAGE_NAME = "Model Trainer stage"

try:
    config = ConfigurationManager()
    data_validation_config = config.get_model_trainer_config()
    data_validation = ModelTrainer(config=data_validation_config)
    data_validation.train()
    
except Exception as e:
    logger.exception(e)
    raise e

[2025-11-18 13:14:11,467: INFO: common :yaml file: config\config.yaml loaded]
[2025-11-18 13:14:11,471: INFO: common :yaml file: params.yaml loaded]
[2025-11-18 13:14:11,479: INFO: common :yaml file: schema.yaml loaded]
[2025-11-18 13:14:11,482: INFO: common :created directory at:artifacts]
alphabeta {'ElasticNet': {'alpha': 0.2, 'l1_ratio': 0.1}}
[2025-11-18 13:14:11,485: INFO: common :created directory at:artifacts]
[2025-11-18 13:14:11,512: ERROR: 3119319871 :[Errno 2] No such file or directory: 'artifacts/model_trainer\\model.joblib']
Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_27852\3119319871.py", line 7, in <module>
    data_validation.train()
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_27852\3990127288.py", line 25, in train
    joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
  File "c:\Users\pc\Desktop\ai\mlops-datascience\venv\lib\site-packages\joblib\numpy_pickle.py", line 599, in dump
    with open(filen

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/model_trainer\\model.joblib'